# CommonLit - Predicting Readability

In this notebook I create an algorithm that predicts the readability of texts from the CommonLit dataset.

I proceed as follows:
1. Extract 3 Test sets from the Training dataset
2. Make 10-fold validation splits using the remaing Training data
3. Apply a RoBERTa tokenizer to each text
4. Apply a CountVectorizer to the tokenized texts
5. Apply a neural network on the "CountVectorized" texts
6. Make 10 predictions for each Test dataset
7. Compute the Mean and Median of all 10 predictions, for each Test dataset

On the 3 Test Datasets (explained in step 1) I get $R^2s$ ranging between 0.85 and 0.95 and $RMSEs$ between 0.3 and 0.8.

However, this notebook cannot be submitted to the CommonLit Prize competition because it requires access to the internet in order to initialize the RoBERTa tokenizer. And to submit a notebook to the CommonLit Prize competition, the notebook must work **without** internet access.

In [1]:
# Import Data

import pandas as pd

Train_Data = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
Test_Data = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
Sample_Submission = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

I implement a Train-Test split to extract 3 Test Data Sets from the Train Data, with 7 observations in each test set.

In [2]:
from sklearn.model_selection import train_test_split

New_Train_Data, New_Test_Data = train_test_split(Train_Data, test_size=0.009, random_state=59)
Test_Data1, Test_Data2 = train_test_split(New_Test_Data, test_size=0.33, random_state=39)
Test_Data1, Test_Data3 = train_test_split(Test_Data1, test_size=0.5, random_state=17)

Initialize the RoBERTa tokenizer.

In [3]:
from transformers import RobertaConfig, RobertaModel

# Initializing a RoBERTa configuration
configuration = RobertaConfig()

# Initializing a model from the configuration
model = RobertaModel(configuration)

# Accessing the model configuration
configuration = model.config

I define a function that:
* does the pre-treatment of all texts
* applies a RoBERTa tokenizer
* applies a CountVectorizer to the previously tokenized texts
* defines a neural network
* returns a prediction for each test dataset

In [4]:
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dropout
from transformers import RobertaTokenizer
import keras


tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def TFIDF_NeuralNetwork_Prediction(Data_Train, Data_Validation, Data_Test, Data_Test1, Data_Test2, Data_Test3):
    
    # Store each excertp into lists: Train_Corpus, Validation_Corpus, Test_Corpus
    
    Data_Train = Data_Train.reset_index()
    Data_Validation = Data_Validation.reset_index()
    Data_Test = Data_Test.reset_index()
    
    Train_Corpus = []
    for j in range(len(Data_Train)):
        bla = str(Data_Train['excerpt'].iloc[j])
        bla = tokenizer(bla)['input_ids']
        Roberta_Sentence = ""
        for h in range(len(bla)):
            Roberta_Sentence = Roberta_Sentence + " " + str(bla[h])
        Train_Corpus.append(Roberta_Sentence)
    
    Validation_Corpus = []
    for j in range(len(Data_Validation)):
        bla = str(Data_Validation['excerpt'].iloc[j])
        bla = tokenizer(bla)['input_ids']
        Roberta_Sentence = ""
        for h in range(len(bla)):
            Roberta_Sentence = Roberta_Sentence + " " + str(bla[h])
        Validation_Corpus.append(Roberta_Sentence)
        
    Test_Corpus = []
    for j in range(len(Data_Test)):
        bla = str(Data_Test['excerpt'].iloc[j])
        bla = tokenizer(bla)['input_ids']
        Roberta_Sentence = ""
        for h in range(len(bla)):
            Roberta_Sentence = Roberta_Sentence + " " + str(bla[h])
        Test_Corpus.append(Roberta_Sentence)
        
    Test_Corpus1 = []
    for j in range(len(Data_Test1)):
        bla = str(Data_Test1['excerpt'].iloc[j])
        bla = tokenizer(bla)['input_ids']
        Roberta_Sentence = ""
        for h in range(len(bla)):
            Roberta_Sentence = Roberta_Sentence + " " + str(bla[h])
        Test_Corpus1.append(Roberta_Sentence)
        
    Test_Corpus2 = []
    for j in range(len(Data_Test2)):
        bla = str(Data_Test2['excerpt'].iloc[j])
        bla = tokenizer(bla)['input_ids']
        Roberta_Sentence = ""
        for h in range(len(bla)):
            Roberta_Sentence = Roberta_Sentence + " " + str(bla[h])
        Test_Corpus2.append(Roberta_Sentence)
        
    Test_Corpus3 = []
    for j in range(len(Data_Test3)):
        bla = str(Data_Test3['excerpt'].iloc[j])
        bla = tokenizer(bla)['input_ids']
        Roberta_Sentence = ""
        for h in range(len(bla)):
            Roberta_Sentence = Roberta_Sentence + " " + str(bla[h])
        Test_Corpus3.append(Roberta_Sentence)
    
    # Fit TF-IDF on the Training_Corpus, then transfrom Validation_Corpus, Test
    
    CV = CountVectorizer(analyzer='word',stop_words= 'english')
    
    CV.fit(Train_Corpus)
    
    CV_Train = CV.transform(Train_Corpus)
    CV_Validation = CV.transform(Validation_Corpus)
    
    CV_Test = CV.transform(Test_Corpus)
    CV_Test1 = CV.transform(Test_Corpus1)
    CV_Test2 = CV.transform(Test_Corpus2)
    CV_Test3 = CV.transform(Test_Corpus3)
    
    # Store TF-IDFs in DataFrames

    CV_DataTrain = pd.DataFrame(data = CV_Train.toarray(),columns = CV.get_feature_names())
    CV_DataValidation = pd.DataFrame(data = CV_Validation.toarray(),columns = CV.get_feature_names())
    
    CV_DataTest = pd.DataFrame(data = CV_Test.toarray(),columns = CV.get_feature_names())
    CV_DataTest1 = pd.DataFrame(data = CV_Test1.toarray(),columns = CV.get_feature_names())
    CV_DataTest2 = pd.DataFrame(data = CV_Test2.toarray(),columns = CV.get_feature_names())
    CV_DataTest3 = pd.DataFrame(data = CV_Test3.toarray(),columns = CV.get_feature_names())
    
    # Build Model
    
    model = keras.models.Sequential([
        keras.layers.Dense(2000, activation="relu", input_shape=CV_Train.shape),       
        keras.layers.Dropout(0.1),                            
        keras.layers.Dense(1000, activation="relu"), 
        keras.layers.Dropout(0.1),                               
        keras.layers.Dense(500, activation="relu"),
        keras.layers.Dropout(0.1), 
        keras.layers.Dense(250, activation="relu"),
        keras.layers.Dropout(0.1), 
        keras.layers.Dense(125, activation="relu"),
        keras.layers.Dropout(0.1), 
        keras.layers.Dense(60, activation="relu"),
        keras.layers.Dropout(0.1), 
        keras.layers.Dense(30, activation="relu"),
        keras.layers.Dropout(0.1), 
        keras.layers.Dense(15, activation="relu"),
        keras.layers.Dropout(0.1), 
        keras.layers.Dense(1)
    ])
    
    # Fit Model on Train data, validate on validation data, and predict on Test_Data
    
    model.compile(loss="mse", optimizer="sgd")
    history = model.fit(CV_DataTrain , Data_Train['target'], epochs=150,
                        validation_data=(CV_DataValidation , Data_Validation['target']))
    
    Predictions = model.predict(CV_DataTest)
    
    Predictions1 = model.predict(CV_DataTest1)
    Predictions2 = model.predict(CV_DataTest2)
    Predictions3 = model.predict(CV_DataTest3)
    
    return Predictions, Predictions1, Predictions2, Predictions3

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

I implement a 10-fold Cross-Validation split, and then run the previously defined function that applies a RoBERTa tokenizer, a CountVectorizer, and then a neural network to make 10 predictions for every text on the 3 *Test sets*.

In [5]:
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

kf = KFold(n_splits=10,random_state=123, shuffle=True)

Split = -1
for train_index, validation_index in kf.split(New_Train_Data):
    Split = Split + 1
    X_Train, X_Validation = New_Train_Data.iloc[train_index], New_Train_Data.iloc[validation_index]
    Test_Data['Prediction_'+str(Split)], Test_Data1['Prediction_'+str(Split)], Test_Data2['Prediction_'+str(Split)], Test_Data3['Prediction_'+str(Split)] = TFIDF_NeuralNetwork_Prediction(X_Train, X_Validation, Test_Data, Test_Data1, Test_Data2, Test_Data3)

Epoch 1/150
79/79 [==============================] - 3s 16ms/step - loss: 1.3635 - val_loss: 0.9806
Epoch 2/150
79/79 [==============================] - 1s 10ms/step - loss: 0.8620 - val_loss: 0.6745
Epoch 3/150
79/79 [==============================] - 1s 8ms/step - loss: 0.6198 - val_loss: 0.6439
Epoch 4/150
79/79 [==============================] - 1s 9ms/step - loss: 0.5219 - val_loss: 0.5530
Epoch 5/150
79/79 [==============================] - 1s 8ms/step - loss: 0.4536 - val_loss: 0.5212
Epoch 6/150
79/79 [==============================] - 1s 9ms/step - loss: 0.3572 - val_loss: 0.5503
Epoch 7/150
79/79 [==============================] - 1s 9ms/step - loss: 0.3295 - val_loss: 0.4883
Epoch 8/150
79/79 [==============================] - 1s 9ms/step - loss: 0.3217 - val_loss: 0.5215
Epoch 9/150
79/79 [==============================] - 1s 9ms/step - loss: 0.2696 - val_loss: 0.5036
Epoch 10/150
79/79 [==============================] - 1s 9ms/step - loss: 0.2489 - val_loss: 0.5019
Epoch 1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/150
79/79 [==============================] - 1s 12ms/step - loss: 1.2723 - val_loss: 0.6830
Epoch 2/150
79/79 [==============================] - 1s 9ms/step - loss: 0.7231 - val_loss: 0.5587
Epoch 3/150
79/79 [==============================] - 1s 9ms/step - loss: 0.5401 - val_loss: 0.4908
Epoch 4/150
79/79 [==============================] - 1s 9ms/step - loss: 0.4593 - val_loss: 0.6545
Epoch 5/150
79/79 [==============================] - 1s 9ms/step - loss: 0.4310 - val_loss: 0.4784
Epoch 6/150
79/79 [==============================] - 1s 9ms/step - loss: 0.3498 - val_loss: 0.4590
Epoch 7/150
79/79 [==============================] - 1s 10ms/step - loss: 0.2861 - val_loss: 0.5099
Epoch 8/150
79/79 [==============================] - 1s 11ms/step - loss: 0.2469 - val_loss: 0.4627
Epoch 9/150
79/79 [==============================] - 1s 9ms/step - loss: 0.2283 - val_loss: 0.4834
Epoch 10/150
79/79 [==============================] - 1s 9ms/step - loss: 0.2131 - val_loss: 0.5056
Epoch 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/150
79/79 [==============================] - 1s 12ms/step - loss: 1.3768 - val_loss: 0.8157
Epoch 2/150
79/79 [==============================] - 1s 9ms/step - loss: 0.8339 - val_loss: 0.5991
Epoch 3/150
79/79 [==============================] - 1s 9ms/step - loss: 0.6907 - val_loss: 0.5180
Epoch 4/150
79/79 [==============================] - 1s 9ms/step - loss: 0.5675 - val_loss: 0.5119
Epoch 5/150
79/79 [==============================] - 1s 9ms/step - loss: 0.4546 - val_loss: 0.5075
Epoch 6/150
79/79 [==============================] - 1s 9ms/step - loss: 0.3974 - val_loss: 0.5034
Epoch 7/150
79/79 [==============================] - 1s 13ms/step - loss: 0.3088 - val_loss: 0.5289
Epoch 8/150
79/79 [==============================] - 1s 11ms/step - loss: 0.3295 - val_loss: 0.5158
Epoch 9/150
79/79 [==============================] - 1s 10ms/step - loss: 0.2845 - val_loss: 0.5711
Epoch 10/150
79/79 [==============================] - 1s 9ms/step - loss: 0.2472 - val_loss: 0.5077
Epoch

...

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/150
79/79 [==============================] - 2s 14ms/step - loss: 1.2826 - val_loss: 0.7742
Epoch 2/150
79/79 [==============================] - 1s 8ms/step - loss: 0.7178 - val_loss: 0.5664
Epoch 3/150
79/79 [==============================] - 1s 8ms/step - loss: 0.5695 - val_loss: 0.4948
Epoch 4/150
79/79 [==============================] - 1s 8ms/step - loss: 0.5108 - val_loss: 0.5037
Epoch 5/150
79/79 [==============================] - 1s 8ms/step - loss: 0.3908 - val_loss: 0.4822
Epoch 6/150
79/79 [==============================] - 1s 8ms/step - loss: 0.3223 - val_loss: 0.6784
Epoch 7/150
79/79 [==============================] - 1s 8ms/step - loss: 0.3038 - val_loss: 0.5120
Epoch 8/150
79/79 [==============================] - 1s 10ms/step - loss: 0.2565 - val_loss: 0.5387
Epoch 9/150
79/79 [==============================] - 1s 8ms/step - loss: 0.2139 - val_loss: 0.4946
Epoch 10/150
79/79 [==============================] - 1s 8ms/step - loss: 0.2395 - val_loss: 0.5056
Epoch 1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/150
79/79 [==============================] - 1s 11ms/step - loss: 1.3920 - val_loss: 0.8302
Epoch 2/150
79/79 [==============================] - 1s 8ms/step - loss: 0.8084 - val_loss: 0.5932
Epoch 3/150
79/79 [==============================] - 1s 8ms/step - loss: 0.6290 - val_loss: 0.5800
Epoch 4/150
79/79 [==============================] - 1s 8ms/step - loss: 0.5388 - val_loss: 0.5463
Epoch 5/150
79/79 [==============================] - 1s 8ms/step - loss: 0.4471 - val_loss: 0.5158
Epoch 6/150
79/79 [==============================] - 1s 8ms/step - loss: 0.3829 - val_loss: 0.5084
Epoch 7/150
79/79 [==============================] - 1s 10ms/step - loss: 0.3342 - val_loss: 0.5249
Epoch 8/150
79/79 [==============================] - 1s 8ms/step - loss: 0.2869 - val_loss: 0.6790
Epoch 9/150
79/79 [==============================] - 1s 8ms/step - loss: 0.2806 - val_loss: 0.7484
Epoch 10/150
79/79 [==============================] - 1s 8ms/step - loss: 0.2210 - val_loss: 0.5097
Epoch 1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


I define functions to calculate evaluation metrics: $R^2$ and $RMSE$.

In [6]:
# Define functions which compute evaluation scores (R2 and RMSE)

import numpy as np

def R2(RealY,PredictedY):
    Top = (RealY - PredictedY)**2
    MeanY = np.ones(len(RealY))
    Bottom = (RealY - MeanY)**2
    R2 = 1 - Top.sum() / Bottom.sum()
    return R2

def RMSE(RealY,PredictedY):
    RMSE = (((RealY - PredictedY)**2).mean())**(1/2)
    return RMSE

Having produced 10 predictions for each text, I create two functions to aggregate the 10 predictions:
* Median of the 10 predictions
* Mean of the 10 predictions

In [7]:
import statistics

def MedianPrediction(DataFrame, nb_predictions):
    MedianPredictions = []
    for i in range(len(DataFrame)):
        Predictions = []
        for j in range(nb_predictions):
            VariableName = 'Prediction_' + str(j)
            Predictions.append(DataFrame[VariableName].iloc[i])
        MedianPredictions.append(statistics.median(Predictions))
    return MedianPredictions


def MeanPrediction(DataFrame, nb_predictions):
    DataFrame['MeanPredictions'] = 0
    for j in range(nb_predictions):
        VariableName = 'Prediction_' + str(j)
        DataFrame['MeanPredictions'] = DataFrame['MeanPredictions']  +  DataFrame[VariableName]
    DataFrame['MeanPredictions'] = DataFrame['MeanPredictions'] / nb_predictions
    MeanPrediction =   DataFrame['MeanPredictions'].to_list()
    return MeanPrediction

I apply the 2 previously defined functions to orginal test data set (which should be used for the submission), and to the 3 test data sets I have created in the second cell of this notebook.

In [8]:
# Apply Mean Prediction and Median Prediction to the 10 predictions of the orginal test set (used to make the submission)

Test_Data['MedianPredictions'] = MedianPrediction(Test_Data, 10)
Test_Data['MeanPredictions'] = MeanPrediction(Test_Data, 10)
Test_Data['FinalPredictions'] = 0.5*Test_Data['MedianPredictions'] + 0.5*Test_Data['MeanPredictions']


# Apply Mean Prediction and Median Prediction to the 10 predictions of the orginal test set (used for preliminary evaluations)

Test_Data1['MedianPredictions'] = MedianPrediction(Test_Data1, 10)
Test_Data1['MeanPredictions'] = MeanPrediction(Test_Data1, 10)
Test_Data1['FinalPredictions'] = 0.5*Test_Data1['MedianPredictions'] + 0.5*Test_Data1['MeanPredictions']


Test_Data2['MedianPredictions'] = MedianPrediction(Test_Data2, 10)
Test_Data2['MeanPredictions'] = MeanPrediction(Test_Data2, 10)
Test_Data2['FinalPredictions'] = 0.5*Test_Data2['MedianPredictions'] + 0.5*Test_Data2['MeanPredictions']


Test_Data3['MedianPredictions'] = MedianPrediction(Test_Data3, 10)
Test_Data3['MeanPredictions'] = MeanPrediction(Test_Data3, 10)
Test_Data3['FinalPredictions'] = 0.5*Test_Data3['MedianPredictions'] + 0.5*Test_Data3['MeanPredictions']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

Print evaluation metrics.

In [9]:
print('Median Predictions - R2 :')
print(" ")
print("Test 1 : ", R2(Test_Data1['target'],Test_Data1['MedianPredictions']))
print("Test 2 : ", R2(Test_Data2['target'],Test_Data2['MedianPredictions']))
print("Test 3 : ", R2(Test_Data3['target'],Test_Data3['MedianPredictions']))
print(" ")
print('Mean Predictions - R2 :')
print(" ")
print("Test 1 : ", R2(Test_Data1['target'],Test_Data1['MeanPredictions']))
print("Test 2 : ", R2(Test_Data2['target'],Test_Data2['MeanPredictions']))
print("Test 3 : ", R2(Test_Data3['target'],Test_Data3['MeanPredictions']))
print(" ")
print('Final Predictions - R2 :')
print(" ")
print("Test 1 : ", R2(Test_Data1['target'],Test_Data1['FinalPredictions']))
print("Test 2 : ", R2(Test_Data2['target'],Test_Data2['FinalPredictions']))
print("Test 3 : ", R2(Test_Data3['target'],Test_Data3['FinalPredictions']))


print(" ")
print(" ")
print('Median Predictions - RMSE :')
print(" ")
print("Test 1 : ", RMSE(Test_Data1['target'],Test_Data1['MedianPredictions']))
print("Test 2 : ", RMSE(Test_Data2['target'],Test_Data2['MedianPredictions']))
print("Test 3 : ", RMSE(Test_Data3['target'],Test_Data3['MedianPredictions']))
print(" ")
print('Mean Predictions - RMSE :')
print(" ")
print("Test 1 : ", RMSE(Test_Data1['target'],Test_Data1['MeanPredictions']))
print("Test 2 : ", RMSE(Test_Data2['target'],Test_Data2['MeanPredictions']))
print("Test 3 : ", RMSE(Test_Data3['target'],Test_Data3['MeanPredictions']))
print(" ")
print('Final Predictions - RMSE :')
print(" ")
print("Test 1 : ", RMSE(Test_Data1['target'],Test_Data1['FinalPredictions']))
print("Test 2 : ", RMSE(Test_Data2['target'],Test_Data2['FinalPredictions']))
print("Test 3 : ", RMSE(Test_Data3['target'],Test_Data3['FinalPredictions']))

Median Predictions - R2 :
 
Test 1 :  0.9039120752254374
Test 2 :  0.9205808162738243
Test 3 :  0.9603847037925102
 
Mean Predictions - R2 :
 
Test 1 :  0.9011667182392858
Test 2 :  0.9208359207994135
Test 3 :  0.9612798985713772
 
Final Predictions - R2 :
 
Test 1 :  0.9026494060085194
Test 2 :  0.9207358612772581
Test 3 :  0.9608688642691503
 
 
Median Predictions - RMSE :
 
Test 1 :  0.6047696177971502
Test 2 :  0.659933678208614
Test 3 :  0.4550290743984726
 
Mean Predictions - RMSE :
 
Test 1 :  0.6133483005892919
Test 2 :  0.6588729302279271
Test 3 :  0.44985850585502757
 
Final Predictions - RMSE :
 
Test 1 :  0.6087302178340598
Test 2 :  0.6592891903070525
Test 3 :  0.4522399451547947


Create the submission sample.

In [10]:
New_Sample_Submission = Sample_Submission.copy()
New_Sample_Submission['target'] = Test_Data['FinalPredictions']

New_Sample_Submission.to_csv('submission.csv',index=False)